# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [37]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [38]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [39]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [40]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [41]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

# import Cluster from cassandra module
from cassandra.cluster import Cluster

# set variable to cassandra istance
cluster = Cluster(['127.0.0.1'])

# set connection
try:
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [42]:
# create a Keyspace named udacity
try:
    stmt = """CREATE KEYSPACE IF NOT EXISTS udacity 
            WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
    session.execute(stmt)
except Exception as e:
    print(e)


#### Set Keyspace

In [43]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [44]:
##  Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# as required to filter set as primary key session_id and item_in_session that garantee also uniqueness on data, no order data needed
try:
    stmt = """DROP TABLE music_library_by_session"""
    session.execute(stmt)
except Exception as e:
    print(e)

try:
    stmt = """CREATE TABLE IF NOT EXISTS music_library_by_session (session_id int, item_in_session int, artist_name varchar, 
              song_title varchar, lenght double, PRIMARY KEY (session_id, item_in_session))"""
    session.execute(stmt)
except Exception as e:
    print(e)
                    

Error from server: code=2200 [Invalid query] message="unconfigured table music_library_by_session"


In [45]:
# set data file event
file = 'event_datafile_new.csv'

# open data file evente and process row by row doing insert to target table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_session (session_id, item_in_session, artist_name, song_title, lenght) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

In [46]:
# set QUERY 1, execute and show the results
try:
    stmt = """SELECT artist_name, song_title, lenght FROM music_library_by_session WHERE session_id = 338 AND item_in_session = 4"""
    rows = session.execute(stmt)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song_title, row.lenght)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [47]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# as required to filter set as composite primary key user_id and session_id to improve overall performance , 
# order data needed on item_in_session column, set it as clustering_column and that with user_id and session_id should guarantees uniqueness on data

try:
    stmt = """DROP TABLE music_library_by_session_user"""
    session.execute(stmt)
except Exception as e:
    print(e)

try:
    stmt = """CREATE TABLE IF NOT EXISTS music_library_by_session_user (user_id int, session_id int, item_in_session int,
              artist_name varchar, song_title varchar, first_name varchar, last_name varchar, 
              PRIMARY KEY ((user_id, session_id), item_in_session))"""
    session.execute(stmt)
except Exception as e:
    print(e)                    

Error from server: code=2200 [Invalid query] message="unconfigured table music_library_by_session_user"


In [48]:
# set data file event
file = 'event_datafile_new.csv'

# open data file evente and process row by row doing insert to target table

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO music_library_by_session_user (user_id, session_id, item_in_session, artist_name, 
                   song_title, first_name, last_name) """
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)
            break

In [49]:
# set QUERY 2, execute and show the results
try:
    stmt = """SELECT artist_name, song_title, first_name, last_name, item_in_session FROM music_library_by_session_user 
              WHERE user_id = 10 AND session_id =182"""
    rows = session.execute(stmt)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name, row.item_in_session)

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


In [50]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# as required to filter set as primary key song_name and user_id to guarantees uniqueness, no order needed 

# drop table
try:
    stmt = """DROP TABLE user_hist"""
    session.execute(stmt)
except Exception as e:
    print(e)

# create table
try:
    stmt = """CREATE TABLE IF NOT EXISTS user_hist (song_title varchar, user_id int, first_name varchar, last_name varchar,  
            PRIMARY KEY (song_title, user_id))"""
    session.execute(stmt)
except Exception as e:
    print(e) 

                    

Error from server: code=2200 [Invalid query] message="unconfigured table user_hist"


In [51]:
# set data file event
file = 'event_datafile_new.csv'

# open data file evente and process row by row doing insert to target table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_hist (song_title, user_id, first_name,
            last_name) """
        query = query + "VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4] ))
        except Exception as e:
            print(e)

In [52]:
# set QUERY 3, execute and show the results
try:
    stmt = """SELECT first_name, last_name FROM user_hist
              WHERE song_title = 'All Hands Against His Own' """
    rows = session.execute(stmt)
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [53]:
## Drop the table before closing out the sessions

In [54]:
try:
    stmt = """DROP TABLE music_library_by_session"""
    session.execute(stmt)
except Exception as e:
    print(e)
try:
    stmt = """DROP TABLE music_library_by_session_user"""
    session.execute(stmt)
except Exception as e:
    print(e)
try:
    stmt = """DROP TABLE user_hist"""
    session.execute(stmt)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [55]:
session.shutdown()
cluster.shutdown()